These are simple use cases for thiis but you can always modify the queries to suit your purpose!

##Install packages

In [ ]:
!pip -q install langchain huggingface_hub transformers sentence_transformers chromadb auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 9.1 MB/s eta 0:00:00


## Import packages

In [ ]:
import os
from langchain.tools import PubmedQueryRun
import os
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader

##Download model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from langchain.llms import HuggingFacePipeline
import torch

model_name = "TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_name)
quantize_config = BaseQuantizeConfig.from_pretrained(model_name)
model = AutoGPTQForCausalLM.from_quantized(model_name,
                                           use_safetensors=True,
                                           model_basename="Wizard-Vicuna-13B-Uncensored-GPTQ-4bit-128g.compat.no-act-order",
                                           device="cuda:0",
                                           use_triton=False, # True or False
                                           quantize_config=quantize_config)

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import torch

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15
)

local_llm = HuggingFacePipeline(pipeline=pipe)

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerN

## Get pubmed data
Replace email with your email address and max_results can be changed as needed. Remember Pubmed APIhas a limit to the number of abstracts you can retrieve

In [ ]:
from fpdf import FPDF
from pymed import PubMed
import json
from datetime import date

# Define a custom serializer function
def default_serializer(o):
    if isinstance(o, date):
        return o.isoformat()
    raise TypeError(f"Object of type {o.__class__.__name__} is not JSON serializable")

# Instantiate the PubMed object
pubmed = PubMed(tool="MyTool", email="email")

# Execute the query
results = pubmed.query("CB1 AND MD simulations", max_results=50)

articles_dict = {}

# Loop through the articles returned by the query
for article in results:
    # Handle potential missing fields in the article
    try:
        title = article.title
    except AttributeError:
        title = "N/A"
    try:
        authors = ", ".join([author['lastname'] if author['lastname'] is not None else 'N/A' for author in article.authors])
    except AttributeError:
        authors = "N/A"
    try:
        abstract = article.abstract
    except AttributeError:
        abstract = "N/A"
    try:
        publication_date = article.publication_date
    except AttributeError:
        publication_date = "N/A"
    try:
        journal = article.journal
    except AttributeError:
        journal = "N/A"

    # Use 'Title' as the key in the articles_dict
    articles_dict[title] = {
        "Authors": authors,
        "Abstract": abstract,
        "Publication Date": publication_date,
        "Journal": journal
    }

# Open the text file
with open('results.txt', 'w') as f:
    # Loop over the articles
    for title, article in articles_dict.items():
        # Write each field to the file
        f.write(f"Title: {title}\n")
        f.write(f"Authors: {article['Authors']}\n")
        f.write(f"Abstract: {article['Abstract']}\n")
        f.write(f"Publication Date: {article['Publication Date']}\n")
        f.write(f"Journal: {article['Journal']}\n")
        f.write("\n")  # Write a newline to separate articles


##Download embeddings

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

model_name = "intfloat/e5-large-v2"

hf = HuggingFaceEmbeddings(model_name=model_name)

## Load and embed results

In [ ]:
#load results
loader = TextLoader('/content/results.txt')

text = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(text)

In [ ]:
texts

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = hf #instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=local_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
qa_chain.combine_documents_chain.llm_chain.prompt.template = '''
You are citation expert, and you are provided with a database composed of Pubchem abstracts,
title, authors, journals and date published. The user can ask you questions based on the data provided.
For example what did Wu, Li, Hua, Liu, Liu, Zhao study?

If the query is not in the provided database, DO NOT ANSWER and do not make anything up!
----------------
{context}

Question: {question}
Helpful Answer:'''

In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    temp_resp = wrap_text_preserve_newlines(llm_response['result'])
    #temp_resp = trim_string(temp_resp)
    print(temp_resp)
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "Who are the authors of MD Simulations Revealing Special Activation Mechanism of Cannabinoid Receptor 1?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Wu, Li, Hua, Liu, Liu, Zhao


Sources:
/content/results.txt
/content/results.txt


In [ ]:
# full example
query = "Summarize the abstract of TRPV1 Activation by Anandamide via a Unique Lipid Pathway?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The article discusses how the TRPV1 receptor can be activated by the endocannabinoid anandamide through a
unique lipid pathway involving a peripheral binding site. This finding expands our understanding of this
receptor and could lead to the development of new drugs targeting it.


Sources:
/content/results.txt
/content/results.txt


In [ ]:
# full example
query = "What is the title of the paper written by Padgett?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The title of the paper written by Padgett is "Functional residues essential for the activation of the CB1
cannabinoid receptor".


Sources:
/content/results.txt
/content/results.txt


In [ ]:
# full example
query = "What is the title of the paper written by Messi?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 There is no publication that matches this description in the given database.


Sources:
/content/results.txt
/content/results.txt


The model gets all answers right including the false question about Lionel Messi the footballer! These are of course very simple queries and you can always modify for your case study